In [ ]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [ ]:
import codecs
from datetime import datetime as dt
import datetime
import math
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly, make_japan_choropleth, \
    make_japan_heatmap
from hokkaidomap import make_hokkaido_choropleth, make_hokkaido_plotly

In [ ]:
from air_registance import AirRegistance

In [ ]:
city2pref = {
   "札幌": "Hokkaido", # 北海道
   "青森": "Aomori", # 青森
   "盛岡": "Iwate", # 岩手
   "仙台": "Miyagi", # 宮城
   "秋田": "Akita", # 秋田
   "山形": "Yamagata", # 山形
   "福島": "Fukushima", # 福島
   "水戸": "Ibaraki", # 茨城
   "宇都宮": "Tochigi", # 栃木
   "前橋": "Gunma", # 群馬
   "秩父": "Saitama", # 埼玉
   "千葉": "Chiba", # 千葉
   "東京": "Tokyo", # 東京
   "横浜": "Kanagawa", # 神奈川
   "新潟": "Niigata", # 新潟
   "富山": "Toyama", # 富山
   "金沢": "Ishikawa", # 石川
   "福井": "Fukui", # 福井
   "甲府": "Yamanashi", # 山梨
   "長野": "Nagano", # 長野
   "岐阜": "Gifu", # 岐阜
   "静岡": "Shizuoka", # 静岡
   "名古屋": "Aichi", # 愛知
   "津": "Mie", # 三重
   "彦根": "Shiga", # 滋賀
   "京都": "Kyoto", # 京都
   "大阪": "Osaka", # 大阪
   "神戸": "Hyogo", # 兵庫
   "奈良": "Nara", # 奈良
   "和歌山": "Wakayama", # 和歌山
   "松江": "Tottori", # 島根
   "鳥取": "Shimane", # 鳥取
   "岡山": "Okayama", # 岡山
   "広島": "Hiroshima", # 広島
   "山口": "Yamaguchi", # 山口
   "徳島": "Tokushima", # 徳島
   "高松": "Kagawa", # 香川
   "松山": "Ehime", # 愛媛
   "高知": "Kochi", # 高知
   "福岡": "Fukuoka", # 福岡
   "佐賀": "Saga", # 佐賀
   "長崎": "Nagasaki", # 長崎
   "熊本": "Kumamoto", # 熊本
   "大分": "Oita", # 大分
   "宮崎": "Miyazaki", # 宮崎
   "鹿児島": "Kagoshima", # 鹿児島
   "那覇": "Okinawa" # 沖縄
}

In [ ]:
cite2hokkai = {
    "札幌"  :  1,# "石狩"
    "函館"  :  2,# "渡島"
    "江差"  :  3,# "檜山"
    "倶知安":  4,# "後志"
    "岩見沢":  5,# "空知"
    "旭川"  :  6,# "上川"
    "留萌"  :  7,# "留萌"
    "稚内"  :  8,# "宗谷"
    "網走"  :  9,# "オホーツク"
    "室蘭"  : 10,# "胆振"
    "浦河"  : 11,# "日高"
    "帯広"  : 12,# "十勝"
    "釧路"  : 13,# "釧路"
    "根室"  : 14,# "根室"
}

In [ ]:
if 16 != dt.now().hour:
    print("nothing to do.")
    if "ipy" in sys.argv[0]:
        pass#exit()
    else:
        sys.exit()

In [ ]:
# 前日のデータ
u = "https://www.data.jma.go.jp/obd/stats/data/mdrr/synopday/data2s.html"
dfs = pd.read_html(u)

In [ ]:
pop_inf = get_populations()

In [ ]:
Fds = np.zeros(47 + 1).tolist()
VHs = np.zeros(47 + 1).tolist()
Fds_hokkaido = np.zeros(len(cite2hokkai) + 1).tolist()
VHs_hokkaido = np.zeros(len(cite2hokkai) + 1).tolist()
Fds[0] = Fds_hokkaido[0] = 5.55e-11
VHs[0] = VHs_hokkaido[0] = 10.0
ar = AirRegistance()
def str2f(x):
    return float(x.replace(')', '').replace(']', ''))
def tPrh(r):
    return str2f(r[5]), str2f(r[1]), str2f(r[11])
def getVH(vp, tc):
    return 18.0 * ((100.0 * vp) / (8.31447 * (273.15 + tc)))
data = np.zeros((1,47))

In [ ]:
cnt = 0
for df in dfs:
    if len(df.columns) == 33:
        for i, row in df.iterrows():
            if row[0] in city2pref:
                cnt += 1
                prefname = city2pref[row[0]]
                t, P, rh = tPrh(row)
                Fd = ar.calc(t=t, P=P, rh=rh)
                vh = getVH(str2f(row[10]), t)
                code = pop_inf[prefname]['code']
                #print(type(Fd), t, P, rh, vh)
                Fds[code] = Fd
                VHs[code] = vh
            if row[0] in cite2hokkai:
                cnt += 1
                t, P, rh = tPrh(row)
                Fd = ar.calc(t=t, P=P, rh=rh)
                vh = getVH(str2f(row[10]), t)
                code = cite2hokkai[row[0]]
                Fds_hokkaido[code] = Fd
                VHs_hokkaido[code] = vh
print("processed {} lines.".format(cnt))

In [ ]:
minFd = min(np.min(Fds), np.min(Fds_hokkaido))
maxFd = max(np.max(Fds), np.min(Fds_hokkaido))
print("Fd min:{} max:{}".format(minFd, maxFd))
Fds[0] = Fds_hokkaido[0] = minFd

In [ ]:
minVH = min(np.min(VHs), np.min(VHs_hokkaido))
maxVH = max(np.max(VHs), np.min(VHs_hokkaido))
print("Fd min:{} max:{}".format(minVH, maxVH))
VHs[0] = VHs_hokkaido[0] = maxVH

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
tw_body_map_fds = "全国 新型コロナ 県別 空気抵抗力(前日平均) ({})".format(today_str)
imgname = 'japan-fds.jpg'
make_japan_choropleth(imgname, tw_body_map_fds, Fds[1:])
make_japan_heatmap(imgname, tw_body_map_fds, Fds, pop_inf)

In [ ]:
tw = get_twitter()

In [ ]:
if False:
    tw_body_map_fds += "低リスク:55p以上? 中リスク:54p以下?, 高リスク:53p以下?"
    tw_body_map_fds += " https://geneasyura.github.io/cov19-hm/airregist.html "
    tweet_with_image(tw, "docs/images/japan-fds.jpg", tw_body_map_fds)

In [ ]:
tw_body_map_vhs = "全国 新型コロナ 県別 絶対湿度(前日平均) ({})".format(today_str)
imgname = 'japan-vh.jpg'   
make_japan_choropleth(imgname, tw_body_map_vhs, VHs[1:])
make_japan_heatmap(imgname, tw_body_map_vhs, VHs, pop_inf)

In [ ]:
tw_body_map_vhs += "高リスク:7g/㎥以下?"
tw_body_map_vhs += " https://geneasyura.github.io/cov19-hm/airregist.html "
if False:
    tweet_with_image(tw, "docs/images/japan-vh.jpg", tw_body_map_vhs)

In [ ]:
tw_body_map_fds = "北海道 新型コロナ 振興局別 空気抵抗力(前日平均) ({})".format(today_str)
imgname = 'hokkaido-map-fds.jpg'
make_hokkaido_choropleth(imgname, tw_body_map_fds, Fds_hokkaido)

In [ ]:
if False:
    tw_body_map_fds += "低リスク:55p以上? 中リスク:54p以下?, 高リスク:53p以下?"
    tw_body_map_fds += " https://geneasyura.github.io/cov19-hm/airregist.html "
    tweet_with_image(tw, "docs/images/hokkaido-map-fds.jpg", tw_body_map_fds)

In [ ]:
tw_body_map_vhs = "北海道 新型コロナ 振興局別 絶対湿度(前日平均) ({})".format(today_str)
imgname = 'hokkaido-map-vh.jpg'
make_hokkaido_choropleth(imgname, tw_body_map_vhs, VHs_hokkaido)

In [ ]:
tw_body_map_vhs += "高リスク:7g/㎥以下?"
tw_body_map_vhs += " https://geneasyura.github.io/cov19-hm/airregist.html "
if False:
    tweet_with_image(tw, "docs/images/hokkaido-map-vh.jpg", tw_body_map_vhs)